<a href="https://colab.research.google.com/github/twumasimb/Image-Restoration-Project-CSCE-5218-/blob/main/image_restoration_thru_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Getting all the models we will be testing
"""
Old-photos to Life
"""
!git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git

"""
GFPGAN
"""
!git clone https://github.com/TencentARC/GFPGAN.git

"""
CycleISP
"""
!git clone https://github.com/swz30/CycleISP.git

Cloning into 'Bringing-Old-Photos-Back-to-Life'...
remote: Enumerating objects: 509, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 509 (delta 3), reused 5 (delta 1), pack-reused 501
Receiving objects: 100% (509/509), 40.90 MiB | 29.05 MiB/s, done.
Resolving deltas: 100% (227/227), done.
Cloning into 'GFPGAN'...
remote: Enumerating objects: 523, done.
remote: Total 523 (delta 0), reused 0 (delta 0), pack-reused 523
Receiving objects: 100% (523/523), 5.39 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (264/264), done.
Cloning into 'CycleISP'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 234 (delta 23), reused 36 (delta 21), pack-reused 193
Receiving objects: 100% (234/234), 12.38 MiB | 29.42 MiB/s, done.
Resolving deltas: 100% (78/78), done.


Bringing Old Images to Life Using VAEs

In [2]:
%cd /content/Bringing-Old-Photos-Back-to-Life/

%cd Face_Enhancement/models/networks/
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../../

%cd Global/detection_models
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../

%cd Face_Detection/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
%cd ../

%cd Face_Enhancement/
!wget https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life/releases/download/v1.0/face_checkpoints.zip
!unzip face_checkpoints.zip
%cd ../

%cd Global/
!wget https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life/releases/download/v1.0/global_checkpoints.zip
!unzip global_checkpoints.zip
%cd ../

!pip install -r requirements.txt

/content/Bringing-Old-Photos-Back-to-Life
/content/Bringing-Old-Photos-Back-to-Life/Face_Enhancement/models/networks
Cloning into 'Synchronized-BatchNorm-PyTorch'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 188 (delta 10), reused 27 (delta 10), pack-reused 161
Receiving objects: 100% (188/188), 47.20 KiB | 1.63 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/Bringing-Old-Photos-Back-to-Life
/content/Bringing-Old-Photos-Back-to-Life/Global/detection_models
Cloning into 'Synchronized-BatchNorm-PyTorch'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 188 (delta 10), reused 27 (delta 10), pack-reused 161
Receiving objects: 100% (188/188), 47.20 KiB | 1.57 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/Bringing-Old-Photos-Back-to-Life
/content/Bringing-O

In [1]:
%cd /content/datasets/
input_folder = "noisy/celeba-100"
%cd /content/datasets/clean/
output_folder = "vae"

import os
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)
output_path = os.path.join(basepath, output_folder)
os.mkdir(output_path)

%cd /content/Bringing-Old-Photos-Back-to-Life
!python run.py --input_folder /content/datasets/noisy/celeba-100/ --output_folder /content/datasets/clean/vae/ --GPU -1

[Errno 2] No such file or directory: '/content/datasets/'
/content
[Errno 2] No such file or directory: '/content/datasets/clean/'
/content
[Errno 2] No such file or directory: '/content/Bringing-Old-Photos-Back-to-Life'
/content
python3: can't open file '/content/run.py': [Errno 2] No such file or directory


In [ ]:
# import os

# %cd /content/datasets/
# input_folder = "noisy/celeba-no-noise"
# basepath = os.getcwd()
# input_path = os.path.join(basepath, input_folder)

# %cd /content/datasets/clean/
# output_folder = "vae-no-noise"
# basepath = os.getcwd()
# output_path = os.path.join(basepath, output_folder)
# os.mkdir(output_path)

# %cd /content/Bringing-Old-Photos-Back-to-Life
# !python run.py --input_folder /content/datasets/noisy/celeba-no-noise/ --output_folder /content/datasets/clean/vae-no-noise/ --GPU -1

In [25]:
!cp -r /content/drive/MyDrive/isp /content/CycleISP/pretrained_models

GFPGANs

In [ ]:
# %cd GFPGAN/
# # We use BasicSR for both training and inference
# !pip install basicsr
# # We use face detection and face restoration helper in the facexlib package
# !pip install facexlib
# # Install other depencencies
# !pip install -r requirements.txt
# !python setup.py develop
# !pip install realesrgan  # used for enhancing the background (non-face) regions
# # Now we use the V1.3 model for the demo
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
# import shutil
# import os
# upload_folder = 'inputs/upload'

# if os.path.isdir(upload_folder):
#     shutil.rmtree(upload_folder)
# os.makedirs(upload_folder, exist_ok=True)
# shutil.move('inputs/whole_imgs/Blake_Lively.jpg', 'inputs/upload/Blake_Lively.jpg')

In [ ]:
# import os

# %cd /content/datasets/
# input_folder = "noisy/celeba-100"
# basepath = os.getcwd()
# input_path = os.path.join(basepath, input_folder)

# %cd /content/datasets/clean/
# output_folder = "gfgan"
# basepath = os.getcwd()
# output_path = os.path.join(basepath, output_folder)
# os.mkdir(output_path)

# %cd /content/GFPGAN/
# !python inference_gfpgan.py -i  /content/datasets/noisy/celeba-100/ -o /content/datasets/clean/gfgan/ -v 1.3 -s 2 --bg_upsampler realesrgan

CycleISP

In [26]:
# Generate Raw and Clean Images from the dataset
%cd /content/CycleISP

/content/CycleISP


I am running CycleISP locally

In [59]:
%cd /content/
# import eval_pred

import math
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as transforms
from torchmetrics import PeakSignalNoiseRatio
import torch.nn.functional as F

def image2tensor(image):
  """
  This function takes the path to an image and convert it into a tensor
  """
  image = Image.open(image)
  transform = transforms.Compose([
      transforms.PILToTensor()
  ])
  img_tensor = transform(image)
  return img_tensor

def psnr(generated, input_img):
  psnr = PeakSignalNoiseRatio()
  preds = torch.tensor(image2tensor(generated))
  target = torch.tensor(image2tensor(input_img))
  if(target.size() != preds.size()):
    preds = F.interpolate(preds, size=target.size())
  psnr_val = psnr(preds, target)
  return psnr_val.to_numpy()

#Evaluate the PSNR of generated image
raw_img = '/content/datasets/noisy/celeba-no-noise/Heng_Fan.png'
gen_img = '/content/datasets/clean/vae/final_output/Heng_Fan.png'
gfpgan_eval = psnr(raw_img, gen_img)

/content


<ipython-input-59-d7121d9bed31>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  preds = torch.tensor(image2tensor(generated))
<ipython-input-59-d7121d9bed31>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(image2tensor(input_img))


ValueError: ignored